[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/kohya_ss-colab/blob/main/kohya_ss_colab.ipynb)

In [ ]:
!git clone -b v1.0 https://github.com/camenduru/kohya_ss
%cd kohya_ss